In [1]:
%load_ext lab_black
import os
from bs4 import BeautifulSoup as bs
import requests
import time
from splinter import browser
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
import pandas as pd

In [2]:
# Save html in txt file in case I need it.
BASE_URL = "https://mars.nasa.gov/news/"

FILE_NAME = "html-requests.txt"

html = requests.get(BASE_URL).text
with open(FILE_NAME, "w+", encoding="utf-8") as f:
    f.write(html)

# ---------------------------------------------------------
FILE = "html-selenium.txt"
FILE_WAIT = "html-selenium-wait.txt"

driver = webdriver.Firefox()
driver.get(BASE_URL)
html = driver.page_source

with open(FILE, "w+", encoding="utf-8") as f:
    f.write(html)

driver.get(BASE_URL)
html = driver.page_source
driver.implicitly_wait(10)
driver.close()

with open(FILE_WAIT, "w+", encoding="utf-8") as f:
    f.write(html)

# NASA Mars News

### Collect the latest News Title and Paragraph Text.


In [3]:
# Since requests.get wasn't bringing the latest news, I used selenium.
url = "https://mars.nasa.gov/news/"
driver = webdriver.Firefox()
driver.get(url)
html = driver.page_source

# The driver opened the page and the soup brought the latest news title
soup = bs(html, "html.parser")
news_title = (soup.find("div", class_="list_text")).find("a").text
print(f"News Title: {news_title}")

# Find the news url
teaser_url = (
    "https://mars.nasa.gov/news/" + soup.find("div", class_="list_text").a["href"]
)

# Soup brought the teaser
r = requests.get(teaser_url)
html = r.text
soup = bs(html, "html.parser")
teaser = (soup.find("div", class_="wysiwyg_content")).find("p").text

print(f"Teaser: {teaser}")


driver.close()

News Title: The MarCO Mission Comes to an End
Teaser: The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.


### JPL Mars Space Images - Featured Image

In [4]:
# Connect to a url to grab NASA's featured Mars image

# Use selenium to navegate to the page with the image
driver = webdriver.Firefox()
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
driver.get(url_image)
time.sleep(1)

elem = driver.find_element_by_id("full_image")
elem.click()
time.sleep(1)

# Navigate to "more info" web site, where the full image URL is
elem = driver.find_element_by_link_text("more info")
elem.click()
time.sleep(1)

# Get the "more info" page URL to parse and find the image in full resolution
url_more_info = driver.current_url

# Use requests and soup to return the image url
r = requests.get(url_more_info)
html = r.text
soup = bs(html, "html.parser")
featured_image_url = (
    "https://www.jpl.nasa.gov" + soup.find("figure", class_="lede").a["href"]
)
print(featured_image_url)

# Close the browser
driver.close()

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17900_hires.jpg


In [5]:
# Mars Weather
# Visit https://twitter.com/marswxreport?lang=en and scrape the latest Mars weather tweet from the page.
# Save the tweet text for the weather report as a variable called `mars_weather`.

# Connect to url for Mars Weather Twitter page

url_twitter = "https://twitter.com/marswxreport?lang=en"

# Create a Beautiful Soup object
r = requests.get(url_twitter)
html = r.text
soup = bs(html, "html.parser")
mars_weather = soup.find_all("div", class_="js-tweet-text-container")

# find the first actual weather tweet

mars_weather = mars_weather[0].text

print(f"The weather today in Mars is: {mars_weather}")

The weather today in Mars is: 
InSight sol 445 (2020-02-26) low -92.8ºC (-135.0ºF) high -12.8ºC (8.9ºF)
winds from the SSE at 5.9 m/s (13.3 mph) gusting to 21.1 m/s (47.3 mph)
pressure at 6.30 hPapic.twitter.com/ihFGVkib6L



In [6]:
################# Mars Facts - use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
#  Use Pandas to convert the data to a HTML table string.

url_facts = "http://space-facts.com/mars/"
mars_facts = pd.read_html(url_facts)

ImportError: lxml not found, please install it

### Mars Hemispheres

In [ ]:
# Obtain high resolution images for each of Mar's hemispheres.
# Save both the image url string for the full resolution hemisphere image,
# and the Hemisphere title containing the hemisphere name.
# Use a Python dictionary to store the data using the keys `img_url` and `title`.
# Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.
# set url and the driver

driver = webdriver.Firefox()
url_hem = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
driver.get(url_hem)
html = driver.page_source

time.sleep(5)

# scrape page into Soup to get the hemispheres names

soup = bs(html, "html.parser")

title_all = soup.find_all("h3")
title = [result.text for result in title_all]
# print(title)

# # hemispheres = driver.find_elements_by_class_name("item")
# title = driver.find_element_by_xpath('//*[@id="product-section"]/div[2]/div/a').text
# print(title)


hemispheres_dict = []

for item in driver.find_elements_by_class_name("item"):
    img_url = driver.find_element_by_class_name("thumb").text
print(img_url)


# for item in driver.find_elements_by_class_name("item"):
#     title = driver.find_element_by_(
#         '//*[@id="product-section"]/div[2]//div/a'
#     ).text
#     img_url = driver.find_element_by_class_name(
#         "thumb"
#     ).text
#     hemispheres_dict.append({"title": title, "img_url": img_url})

# # (
# #     soup.find("div", class_="collapsible results").find("div", class_="item").a["href"]
# # )
# print(hemispheres_dict)


# hemisphere_image_urls = []


# Close the browser
driver.close()